In [1]:
from elasticsearch import Elasticsearch

In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
es_client= Elasticsearch('http://localhost:9200')

In [4]:
es_client.info()

ObjectApiResponse({'name': '265c4b614f25', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'jpJ8o9dhSrKJviPKsjtZ-w', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [5]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            # "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [6]:
from tqdm.auto import tqdm

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|████████████████████████████████████████████████████████| 948/948 [00:20<00:00, 46.55it/s]


In [65]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)
    # print("RESPONSE: ", response)
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    return result_docs

In [94]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT:
    {context}
    """.strip()   

    context_template = """
    Q: {question}
    A: {text}
    """.strip()

    context = ""

    for doc in search_results:
        context = context + context_template.format(question=doc['question'], text=doc['text']).strip() + "\n\n"
    print("CONTEXT:", context)
    prompt = prompt_template.format(question=query, context=context).strip()
    print("PROMPT:", prompt)
    return prompt 

In [95]:
query = "How do I execute a command in a running docker container?"
search_results = elastic_search(query)
prompt = build_prompt(query, search_results)

CONTEXT: Q: How do I debug a docker container?
    A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
    A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
    A: You can copy files from your local machine into a Docker container using the docker cp co

In [96]:
prompt_length = len(prompt)

In [97]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

In [98]:
token_total = encoding.encode(prompt)

In [99]:
token_count = len(token_total)

In [100]:
token_count

331

In [102]:
token_list = []
for token in token_total:
    token_list.append(encoding.decode_single_token_bytes(token))

In [103]:
token_list

[b"You're",
 b' a',
 b' course',
 b' teaching',
 b' assistant',
 b'.',
 b' Answer',
 b' the',
 b' QUESTION',
 b' based',
 b' on',
 b' the',
 b' CONT',
 b'EXT',
 b' from',
 b' the',
 b' FAQ',
 b' database',
 b'.\n',
 b'   ',
 b' Use',
 b' only',
 b' the',
 b' facts',
 b' from',
 b' the',
 b' CONT',
 b'EXT',
 b' when',
 b' answering',
 b' the',
 b' QUESTION',
 b'.\n',
 b'    \n',
 b'   ',
 b' QUESTION',
 b':',
 b' How',
 b' do',
 b' I',
 b' execute',
 b' a',
 b' command',
 b' in',
 b' a',
 b' running',
 b' docker',
 b' container',
 b'?\n',
 b'    \n',
 b'   ',
 b' CONT',
 b'EXT',
 b':\n',
 b'   ',
 b' Q',
 b':',
 b' How',
 b' do',
 b' I',
 b' debug',
 b' a',
 b' docker',
 b' container',
 b'?\n',
 b'   ',
 b' A',
 b':',
 b' Launch',
 b' the',
 b' container',
 b' image',
 b' in',
 b' interactive',
 b' mode',
 b' and',
 b' overriding',
 b' the',
 b' entry',
 b'point',
 b',',
 b' so',
 b' that',
 b' it',
 b' starts',
 b' a',
 b' bash',
 b' command',
 b'.\n',
 b'docker',
 b' run',
 b' -',
 b'